# Chapter 5: Query Optimization & Cost Control – Lab

In this lab, you'll experiment with query performance in Snowflake.  
You’ll learn how projection, filtering, result caching, and warehouse resizing affect performance and cost.

**Pre-requisite**: Ensure `LAB_DB` and the tables `CUSTOMERS`, `SUBSCRIPTIONS`, and `BILLING` are already created.


## 1️⃣ Create & Use Dedicated Warehouse

In [ ]:
-- Use SYSADMIN role and set context
USE ROLE SYSADMIN;
USE DATABASE lab_db;
USE SCHEMA public;
ALTER SESSION SET QUERY_TAG = 'ch5_qo_lab';

-- Create a dedicated lab warehouse
CREATE OR REPLACE WAREHOUSE lab_qo_wh
  WAREHOUSE_SIZE = 'SMALL'
  AUTO_SUSPEND = 60
  AUTO_RESUME = TRUE
  INITIALLY_SUSPENDED = TRUE
  COMMENT = 'Warehouse for query optimization lab';

-- Use the warehouse
USE WAREHOUSE lab_qo_wh;

## 2️⃣ Worst Case: No Projection or Filter

In [ ]:
-- Select * from all 3 tables with joins
SELECT *
FROM   customers c
JOIN   subscriptions s ON c.customer_id = s.customer_id
JOIN   billing b ON s.subscription_id = b.subscription_id
LIMIT 50;

## 3️⃣ Add Column Projection

In [ ]:
SELECT
    c.customer_id,
    c.name,
    b.amount_due
FROM   customers c
JOIN   subscriptions s ON c.customer_id = s.customer_id
JOIN   billing b ON s.subscription_id = b.subscription_id
LIMIT 50;

## 4️⃣ Add Filter and Aggregation

In [ ]:
SELECT
    c.customer_id,
    c.name,
    SUM(b.amount_due) AS total_due
FROM   customers c
JOIN   subscriptions s ON c.customer_id = s.customer_id
JOIN   billing b ON s.subscription_id = b.subscription_id
WHERE  b.billing_date >= '2024-01-01'
GROUP BY c.customer_id, c.name
ORDER BY total_due DESC
LIMIT 10;

## 5️⃣ Re-run to Demonstrate Result Cache

In [ ]:
-- Expect sub-second result due to caching
SELECT
    c.customer_id,
    c.name,
    SUM(b.amount_due) AS total_due
FROM   customers c
JOIN   subscriptions s ON c.customer_id = s.customer_id
JOIN   billing b ON s.subscription_id = b.subscription_id
WHERE  b.billing_date >= '2024-01-01'
GROUP BY c.customer_id, c.name
ORDER BY total_due DESC
LIMIT 10;

## 6️⃣ Preview Query Plan with EXPLAIN

In [ ]:
EXPLAIN USING TEXT
SELECT
    c.customer_id,
    SUM(b.amount_due) AS total_due
FROM   customers c
JOIN   subscriptions s ON c.customer_id = s.customer_id
JOIN   billing b ON s.subscription_id = b.subscription_id
WHERE  b.billing_date >= '2024-01-01'
GROUP BY c.customer_id;

## 7️⃣ Resize Warehouse to MEDIUM

In [ ]:
ALTER WAREHOUSE lab_qo_wh SET WAREHOUSE_SIZE = MEDIUM;
ALTER WAREHOUSE lab_qo_wh SUSPEND;
ALTER WAREHOUSE lab_qo_wh RESUME;

## 8️⃣ Re-run Aggregation (Expect Faster Runtime)

In [ ]:
SELECT
    c.customer_id,
    c.name,
    SUM(b.amount_due) AS total_due
FROM   customers c
JOIN   subscriptions s ON c.customer_id = s.customer_id
JOIN   billing b ON s.subscription_id = b.subscription_id
WHERE  b.billing_date >= '2024-01-01'
GROUP BY c.customer_id, c.name
ORDER BY total_due DESC
LIMIT 10;

## 9️⃣ Optional: Cost Monitoring via Usage History

In [ ]:
-- Must be ACCOUNTADMIN or have USAGE rights
-- This may fail in restricted environments
SELECT
    warehouse_name,
    SUM(credits_used_cloud_services + credits_used_computation) AS total_credits
FROM SNOWFLAKE.INFORMATION_SCHEMA.WAREHOUSE_METERING_HISTORY
WHERE warehouse_name = 'LAB_QO_WH'
  AND start_time >= DATEADD(DAY, -1, CURRENT_TIMESTAMP())
GROUP BY warehouse_name;

## 🔚 Clean Up

In [ ]:
ALTER WAREHOUSE lab_qo_wh SUSPEND;
-- DROP WAREHOUSE IF EXISTS lab_qo_wh;  -- Optional
ALTER SESSION UNSET QUERY_TAG;